In [ ]:
!pip install nltk pandas tensorflow


In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('/content/train.csv')

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.shape

(20800, 5)

In [ ]:
df.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [ ]:
#dropping nan values
df=df.dropna()

In [ ]:
df.isnull().sum()

,0
id,0
title,0
author,0
text,0
label,0


In [ ]:
#Get the independent feature
X=df.drop('label',axis=1)

In [ ]:
#Get the dependent feature
y=df['label']

In [ ]:
X.shape

(18285, 4)

In [ ]:
y.shape

(18285,)

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.18.0'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
#vocabulary size
voc_size=5000

one hot representation

In [ ]:
messages=X.copy()

In [ ]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [ ]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Data preprocessing

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [ ]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[399, 4742, 3464, 4080, 1822, 3004, 2632, 3885, 533, 4100],
 [3213, 1818, 4425, 2457, 4441, 105, 4347],
 [727, 2878, 2621, 547],
 [506, 2713, 2435, 2996, 4191, 715],
 [2650, 4441, 4649, 4000, 4063, 4465, 4441, 2081, 2860, 2668],
 [3476,
  1750,
  1942,
  782,
  2880,
  3556,
  563,
  3137,
  1026,
  1067,
  2444,
  3549,
  1142,
  3617,
  4347],
 [3036, 1198, 1436, 238, 353, 877, 3374, 351, 3073, 1331, 992],
 [3174, 66, 4690, 2660, 470, 4484, 3556, 3535, 3073, 1331, 992],
 [2628, 2572, 132, 4034, 1286, 3542, 1772, 1223, 3556, 4576],
 [1843, 2883, 3635, 708, 4984, 4767, 3308, 187],
 [2919, 237, 966, 1999, 2746, 3869, 3602, 3233, 4208, 249, 939],
 [2996, 3547, 1822, 3542, 3556, 470],
 [244, 1828, 2734, 588, 1573, 97, 4380, 4143, 3998],
 [1631, 2428, 1082, 1367, 1040, 3948, 2861, 3073, 1331, 992],
 [3650, 3493, 3304, 448, 3128, 3073, 1331, 992],
 [3589, 1811, 2912, 890, 4899, 3424, 585, 4772, 2291, 2473],
 [2521, 3414, 1818],
 [4261, 1315, 2750, 2530, 3556, 973, 593, 4347],
 [2079, 2770,

In [ ]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [ ]:
onehot_repr[1]

[3213, 1818, 4425, 2457, 4441, 105, 4347]

Embedding Representation

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3885  533 4100]
 [   0    0    0 ... 4441  105 4347]
 [   0    0    0 ... 2878 2621  547]
 ...
 [   0    0    0 ... 3073 1331  992]
 [   0    0    0 ... 2044 4341 1170]
 [   0    0    0 ... 2463 2424 2854]]


In [ ]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 3213, 1818, 4425, 2457, 4441,  105, 4347], dtype=int32)

Creating model

In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
print(model.summary())

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
len(embedded_docs),y.shape

(18285, (18285,))

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

Model Training

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 11s 40ms/step - accuracy: 0.7728 - loss: 0.4349 - val_accuracy: 0.9163 - val_loss: 0.1931
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - accuracy: 0.9443 - loss: 0.1440 - val_accuracy: 0.9130 - val_loss: 0.2452
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.9597 - loss: 0.1101 - val_accuracy: 0.9185 - val_loss: 0.2074
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 37ms/step - accuracy: 0.9722 - loss: 0.0807 - val_accuracy: 0.9170 - val_loss: 0.2391
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - accuracy: 0.9788 - loss: 0.0622 - val_accuracy: 0.9157 - val_loss: 0.2575
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - accuracy: 0.9813 - loss: 0.0567 - val_accuracy: 0.9125 - val_loss: 0.2736
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 43ms/step - accuracy: 0.9867 - loss: 0.0435 - val_accuracy: 0.9062 - val_loss: 0.3223
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 9s 35ms/step - accuracy: 0.9905 - loss: 0.0324 - val_a

Performance Metrics and Accuracy

In [ ]:
y_pred=model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step


In [ ]:
y_pred=np.where(y_pred > 0.5, 1,0) ##AUC ROC Curve

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred)

array([[3117,  302],
       [ 237, 2379]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9106876553438277

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.89      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

